## Table of Contents
0. Project Overview
1. Convert Tabluar Data to SQL DB
2. Create NLP Request 
3. Convert NLP commands to SQL queries using OpenAI API
4. Input SQL Query into DB

## 0. Project Overview

This project aims to generate and convert natural language input into SQL queries.

As companies increasingly recognize the importance of utilizing and collecting data, there has been an emphasis on enhancing data literacy capabilities to extract and understand data.

The goal is to provide SQL queries through the OpenAI API, enabling non-developers such as PMs, planners, and product owners, who have a good understanding of data but have never written queries themselves, to sufficiently search and utilize information.

For example,
It is common to rely on developers or data analysts who can handle SQL whenever one wants to check data(e.g., PMs, marketing, sales, non-developers, executives, etc.)
- Top 5 countries or cities where the product sells the best
- Which region contributes the most to monthly sales

## 1. Convert Tabluar Data to SQL DB

### 1-1. Install required libraries

In [ ]:
import pandas as pd
import os
from openai import OpenAI

### 1-2. Call Data

In [3]:
df = pd.read_excel("./datasets/AdidasSalesdata.xlsx")
df.head(5)

,Retailer,Retailer ID,Invoice Date,Region,State,City,Gender Type,Product Category,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2021-10-26,Northeast,Pennsylvania,Philadelphia,Men,Apparel,55,125,68750.0,24062.5,0.35,Outlet
1,Foot Locker,1185732,2021-10-27,Northeast,Pennsylvania,Philadelphia,Women,Apparel,45,225,101250.0,30375.0,0.30,Outlet
2,Foot Locker,1185732,2021-10-28,Northeast,Pennsylvania,Philadelphia,Men,Street Footwear,45,475,213750.0,117562.5,0.55,Outlet
3,Foot Locker,1185732,2021-10-29,Northeast,Pennsylvania,Philadelphia,Men,Athletic Footwear,45,125,56250.0,19687.5,0.35,Outlet
4,Foot Locker,1185732,2021-10-30,Northeast,Pennsylvania,Philadelphia,Women,Street Footwear,35,175,61250.0,24500.0,0.40,Outlet


In [4]:
df.columns

Index(['Retailer', 'Retailer ID', 'Invoice Date', 'Region', 'State', 'City',
       'Gender Type', 'Product Category', 'Price per Unit', 'Units Sold',
       'Total Sales', 'Operating Profit', 'Operating Margin', 'Sales Method'],
      dtype='object')

In [5]:
df.columns = [col.replace(' ', '_') for col in df.columns]
df.columns

Index(['Retailer', 'Retailer_ID', 'Invoice_Date', 'Region', 'State', 'City',
       'Gender_Type', 'Product_Category', 'Price_per_Unit', 'Units_Sold',
       'Total_Sales', 'Operating_Profit', 'Operating_Margin', 'Sales_Method'],
      dtype='object')

### 1-2.1 Check Data

In [26]:
# the most basic way to check data using groupby
# sum of sales by region
df.groupby("Region").sum(["Total_Sales"])["Total_Sales"].sort_values(ascending=False)

Region
West         269943182.0
Northeast    186324067.0
Southeast    163171236.0
South        144663181.0
Midwest      135800459.0
Name: Total_Sales, dtype: float64

### 1-3. Generate SQL Database

In [7]:
from sqlalchemy import create_engine, text

In [8]:
temp_db = create_engine("sqlite:///:memory:", echo=True)

In [9]:
data = df.to_sql(name="Sales", con=temp_db)

2024-07-15 19:42:22,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 19:42:22,675 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-07-15 19:42:22,676 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 19:42:22,678 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-07-15 19:42:22,678 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 19:42:22,682 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"Retailer" TEXT, 
	"Retailer_ID" BIGINT, 
	"Invoice_Date" DATETIME, 
	"Region" TEXT, 
	"State" TEXT, 
	"City" TEXT, 
	"Gender_Type" TEXT, 
	"Product_Category" TEXT, 
	"Price_per_Unit" BIGINT, 
	"Units_Sold" BIGINT, 
	"Total_Sales" FLOAT, 
	"Operating_Profit" FLOAT, 
	"Operating_Margin" FLOAT, 
	"Sales_Method" TEXT
)


2024-07-15 19:42:22,683 INFO sqlalchemy.engine.Engine [no key 0.00086s] ()
2024-07-15 19:42:22,684 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Sales_index" ON "Sales" ("index")
2024-07-15 19:42:22,685 IN

In [10]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT Retailer, State, City, Total_Sales, Operating_Profit from Sales ORDER BY Total_Sales DESC LIMIT 1"))

2024-07-15 19:42:23,112 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 19:42:23,114 INFO sqlalchemy.engine.Engine SELECT Retailer, State, City, Total_Sales, Operating_Profit from Sales ORDER BY Total_Sales DESC LIMIT 1
2024-07-15 19:42:23,116 INFO sqlalchemy.engine.Engine [generated in 0.00360s] ()
2024-07-15 19:42:23,119 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
result.all()

[('Walmart', 'Florida', 'Orlando', 825000.0, 371250.0)]

## 2. NLP Request

Helper functions to convert natural language commands into SQL statements

Required Functions
- **table_definition_prompt**: function that informs the structure of the table when using OpenAI API
- **prompt_input**: function that retrieves natural language commands entered from the user
- **handle_response**: function to retrieve prompt result value for API call

### 2-1. Function that informs the structure of the table when using OpenAI API

In [13]:
# GPT한테 우리가 다루는 데이터, 테이블이 어떤 구조인지 알려주는 함수

def table_definition_prompt(df):
    prompt = """Given the following sqlite SQL definition, write queries based on the request
                \n### sqlite SQL table, with its properties:
    #
    # Sales({})
    #
    """.format(",".join(str(x) for x in df.columns))

    return prompt

In [14]:
print(table_definition_prompt(df))

Given the following sqlite SQL definition, write queries based on the request
                
### sqlite SQL table, with its properties:
    #
    # Sales(Retailer,Retailer_ID,Invoice_Date,Region,State,City,Gender_Type,Product_Category,Price_per_Unit,Units_Sold,Total_Sales,Operating_Profit,Operating_Margin,Sales_Method)
    #
    


### 2-2. Function that retrieves natural language commands entered from the user

In [15]:
# 사용자로 부터 어떤걸 확인하고 싶은지 받는 내용의 함수

def prompt_input():
    nlp_text = input("질의 하고자하는 내용을 입력해주세요")
    return nlp_text

In [16]:
nlp_text = prompt_input()

In [17]:
full_prompt = str(table_definition_prompt(df)) + str(nlp_text)
full_prompt

'Given the following sqlite SQL definition, write queries based on the request\n                \n### sqlite SQL table, with its properties:\n    #\n    # Sales(Retailer,Retailer_ID,Invoice_Date,Region,State,City,Gender_Type,Product_Category,Price_per_Unit,Units_Sold,Total_Sales,Operating_Profit,Operating_Margin,Sales_Method)\n    #\n    '

### 2-3. Function to retrieve prompt result value for API call

In [24]:
# 필요한 부분만 추출

def handle_response(response):
    query = response.choices[0].message.content.strip()

    if not query.upper().startswith("SELECT"):
        query = "SELECT" + query
    if not query.endswith(";"):
        query += ";"
    return query

## 3. Convert NLP commands to SQL queries using OpenAI API

### 3.1 OpenAI setting and key input to use API

In [25]:
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

### 3.2 API Call

- Model: Specifiy the model to use, you can select a specific model such as `gpt-3.5-turbo`.
- Message: A list of messages that form the context of the conversations. It can include system messages and user messages, with each message hainv a role and content
	- `system`: Sets the assistant's behavoir by providing instructions.
	- `user`: Represents the user's request or comment that the assistant should respond to
- Other adjustable parameters:
- `max_token`: Controls the maximum length of the generated text. Useful for limiting the response length in conversational models.
- `temperature`: Adjusts the diversity and creativity of the generated response. A lower value will result in more deterministic and predictable responses, while a higher value generates more creative and diverse responses.
- `stop`: Specifies a string or array of strings where the generation should stop. This is useful for ending the response at a specific point.

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that generates SQL queries based on the given SQLite table definition and a natural language request. The query should start with 'SELECT' and ends with a semi-colon(;)."},
        {"role": "user", "content": f"A query to answer: {full_prompt}"},
    ],
    max_tokens=200,
    temperature=1.0,
    stop=None,
)

In [ ]:
response

In [ ]:
print(handle_response(response))

## 4. Input SQL Query into DB

In [ ]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

In [ ]:
result.all()